In [1]:
import argparse
import os
from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
import uvicorn
from threading import Thread
from sse_starlette.sse import EventSourceResponse
import torch

In [2]:
torch.cuda.is_available()

True

参数设置

In [3]:
class Args:
    def __init__(self, base_model=None, lora_model=None, tokenizer_path=None, gpus="0",
                 load_in_8bit=False, load_in_4bit=False, only_cpu=False, alpha="1.0"):
        self.base_model = base_model
        self.lora_model = lora_model
        self.tokenizer_path = tokenizer_path
        self.gpus = gpus
        self.load_in_8bit = load_in_8bit
        self.load_in_4bit = load_in_4bit
        self.only_cpu = only_cpu
        self.alpha = alpha

# 在这里设置你的参数
args = Args(
    base_model="/media/vkeilo/game/github_project/model_datas/chinese-alpaca-2-7b-hf",
    lora_model=None,  # 或保持为None，根据你的需要
    tokenizer_path=None,
    gpus="0,1",
    load_in_8bit=False,
    load_in_4bit=False,
    only_cpu=True,
    alpha="1.0"
)

In [4]:
if args.only_cpu is True:
    args.gpus = ""
    if args.load_in_8bit or args.load_in_4bit:
        raise ValueError("Quantization is unavailable on CPU.")
if args.load_in_8bit and args.load_in_4bit:
    raise ValueError("Only one quantization method can be chosen for inference. Please check your arguments")
os.environ["CUDA_VISIBLE_DEVICES"] = args.gpus

import torch
import torch.nn.functional as F
from transformers import (
    LlamaForCausalLM,
    LlamaTokenizer,
    GenerationConfig,
    TextIteratorStreamer,
    BitsAndBytesConfig
)
import sys

/home/vkeilo/miniconda3/envs/llama/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# openai_api_server.py 的父目录
sys.path.append('/media/vkeilo/game/github_project/Chinese-LLaMA-Alpaca-2/scripts')
sys.path.append('/media/vkeilo/game/github_project/Chinese-LLaMA-Alpaca-2/scripts/openai_server_demo')

In [6]:
from attn_and_long_ctx_patches import apply_attention_patch, apply_ntk_scaling_patch

apply_attention_patch(use_memory_efficient_attention=True)
apply_ntk_scaling_patch(args.alpha)

from openai_api_protocol import (
    ChatCompletionRequest,
    ChatCompletionResponse,
    ChatMessage,
    ChatCompletionResponseChoice,
    CompletionRequest,
    CompletionResponse,
    CompletionResponseChoice,
    EmbeddingsRequest,
    EmbeddingsResponse,
    ChatCompletionResponseStreamChoice,
    DeltaMessage,
)

load_type = torch.float16
if torch.cuda.is_available():
    device = torch.device(0)
else:
    device = torch.device("cpu")
if args.tokenizer_path is None:
    args.tokenizer_path = args.lora_model
    if args.lora_model is None:
        args.tokenizer_path = args.base_model
tokenizer = LlamaTokenizer.from_pretrained(args.tokenizer_path, legacy=True)
if args.load_in_4bit or args.load_in_8bit:
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=args.load_in_4bit,
        load_in_8bit=args.load_in_8bit,
        bnb_4bit_compute_dtype=load_type,
    )
base_model = LlamaForCausalLM.from_pretrained(
    args.base_model,
    torch_dtype=load_type,
    low_cpu_mem_usage=True,
    device_map='auto' if not args.only_cpu else None,
    load_in_4bit=args.load_in_4bit,
    load_in_8bit=args.load_in_8bit,
    quantization_config=quantization_config if (args.load_in_4bit or args.load_in_8bit) else None
    # vkeilo add it
    # output_hidden_states=True
)

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565


USE_MEM_EFF_ATTENTION:  True
STORE_KV_BEFORE_ROPE: False
Apply NTK scaling with ALPHA=1.0
The value of scaling factor will be read from model config file, or set to 1.


Loading checkpoint shards: 100%|██████████| 2/2 [00:03<00:00,  1.97s/it]


#### 查看base模型架构

In [7]:
print(base_model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(55296, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNo

In [8]:
# 以k矩阵为例，所取出的128*4096对应 k向量维度*hidden_size 与attention论文中矩阵中的k矩阵为转置关系
# o矩阵不同，所取出的为 hidden_size*（32*128）的矩阵
k_matrixes = torch.empty(0)
q_matrixes = torch.empty(0)
v_matrixes = torch.empty(0)
o_matrixes = torch.empty(0)
for layer in base_model.model.layers:
    tmp_k_matrixes = layer.self_attn.k_proj.weight.data
    tmp_q_matrixes = layer.self_attn.q_proj.weight.data
    tmp_v_matrixes = layer.self_attn.v_proj.weight.data
    tmp_o_matrixes = layer.self_attn.o_proj.weight.data
    tmp_k_matrixes = tmp_k_matrixes.reshape(1,32,128,4096)
    tmp_q_matrixes = tmp_q_matrixes.reshape(1,32,128,4096)
    tmp_v_matrixes = tmp_v_matrixes.reshape(1,32,128,4096)
    tmp_o_matrixes = tmp_o_matrixes.reshape(1,4096,32,128).permute(0, 2, 1, 3)
    k_matrixes = torch.cat((k_matrixes, tmp_k_matrixes), dim=0)
    q_matrixes = torch.cat((q_matrixes, tmp_q_matrixes), dim=0)
    v_matrixes = torch.cat((v_matrixes, tmp_v_matrixes), dim=0)
    o_matrixes = torch.cat((o_matrixes, tmp_o_matrixes), dim=0)


注意，在论文《Attention is all you need》中，$score = qk^T$的前提是$q_1 = x_1W_Q$,
而在论文《A Mathematical Framework for Transformer Circuits》中，$w_{QK}=W_Q^TW_K$是因为$k_i=W_Kx_i$
两篇论文的k,q,v矩阵互为转置关系，代码以后者为基准。
W_Q矩阵的行数为128（中间向量维度），列数为4096（hidden_size）

In [66]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
import torch
import numpy as np
from tqdm import tqdm

# 构建32*32的列表，对应32层下的32个注意力头
attention_heads_list = np.array([[{} for _ in range(32)] for _ in range(32)])

# 遍历每一层和每个注意力头
for layer_idx in tqdm(range(32), desc='层'):
    for head_idx in tqdm(range(32), desc='头', leave=False):
        # 提取对应层和头的q和k矩阵
        q_matrix = q_matrixes[layer_idx, head_idx, :, :].reshape(128, 4096).to(device)
        k_matrix = k_matrixes[layer_idx, head_idx, :, :].reshape(128, 4096).to(device)
        
        # 将numpy数组转换为torch张量，并将其移动到GPU上
        # q_matrix = q_matrix.clone().detach()
        # k_matrix = k_matrix.clone().detach()
        
        # 计算q^Tk，注意这里的q和k都是已经reshape过的，所以需要转置k来进行矩阵乘法
        q_k_product = torch.matmul(q_matrix.t(), k_matrix)
        
        # 计算特征值和特征向量
        eigenvalues, eigenvectors = torch.linalg.eig(q_k_product)
        print(f'layer{layer_idx} head {head_idx} processed')
        # 填充列表,保存在内存上
        attention_heads_list[layer_idx][head_idx] = {
            '特征值': eigenvalues.to('cpu').detach().numpy(),
            '特征向量': eigenvectors.permute(1,0).to('cpu').detach().numpy()
        }
        # 清理不需要的显存占用
        

层:   0%|          | 0/32 [00:00<?, ?it/s]

layer0 head 0 processed


layer0 head 1 processed


layer0 head 2 processed


layer0 head 3 processed


...

layer0 head 30 processed


层:   3%|▎         | 1/32 [05:33<2:52:05, 333.09s/it]

layer0 head 31 processed


层:   3%|▎         | 1/32 [05:43<2:57:19, 343.22s/it]


KeyboardInterrupt: 

In [ ]:
# 保存array变量attention_heads_list到文件中
with open('datas/QK_array.npy', 'wb') as f:
    np.save(f, attention_heads_list)


In [15]:
tmp_q_matrix = q_matrixes[0, 0, :, :].reshape(128, 4096)
tmp_k_matrix = k_matrixes[0, 0, :, :].reshape(128, 4096)
tq = tmp_q_matrix.clone().detach()
tk = tmp_q_matrix.clone().detach()

In [49]:
mat = torch.matmul(tq.t(),tk)

In [50]:
ta,tb = torch.linalg.eig(mat)

In [53]:
ta.shape

torch.Size([4096])

In [48]:
# 取第一个特征值和特征向量
first_eigenvalue = ta[:, 0]
first_eigenvector = tb[:, 0]

# 验证 Ax = λx
Ax = torch.matmul(mat, first_eigenvector)
Ax_over_lambda = Ax / first_eigenvalue

# 打印验证结果
print("验证结果:", torch.allclose(Ax_over_lambda, first_eigenvector))

# 打印特征值和特征向量
print("第一个特征值:", first_eigenvalue)
print("第一个特征向量:", first_eigenvector)

IndexError: too many indices for tensor of dimension 1

In [59]:
tb[:,0].shape

torch.Size([4096])

In [54]:
lx = ta[0]*tb[:,0]

In [60]:
tb[:,0]

tensor([-0.0082+0.j, -0.0201+0.j,  0.0175+0.j,  ...,  0.0108+0.j,  0.0203+0.j,
        -0.0167+0.j])

In [61]:
lx

tensor([-14.1930+0.j, -34.7486+0.j,  30.3795+0.j,  ...,  18.7687+0.j,  35.1911+0.j,
        -28.8638+0.j])

In [63]:
ax

tensor([-14.1924, -34.7503,  30.3795,  ...,  18.7687,  35.1912, -28.8638])

In [62]:
ax = torch.matmul(mat,tb[:,0].real)

In [42]:
print(mat.shape)
print(tb[0].shape)

torch.Size([4096, 4096])
torch.Size([4096])


In [40]:
tb[0].shape

torch.Size([4096])